In [11]:
# FULL NEPAL LANDSLIDE SUSCEPTIBILITY

# Install and Import Necessary Modules
!pip install joblib pandas numpy --quiet

import pandas as pd
import numpy as np
import joblib
from google.colab import drive

# Configuration & Initialization

drive.mount("/content/drive")

# Paths
MODEL_PATH = "/content/drive/MyDrive/final_project/model_landslide/landslide_xgboost.pkl"

INPUT_DATA_PATH = "/content/drive/MyDrive/final_project/dataset_landslide/dataB.csv"
OUTPUT_PATH = "/content/drive/MyDrive/final_project/landslide_prediction/dataB_prediction(test)_ver4.csv"

# Load Data
print(" Loading trained model and input data...")
model = joblib.load(MODEL_PATH)
df_predict = pd.read_csv(INPUT_DATA_PATH)


# Rename columns
df_predict = df_predict.rename(columns={'mean_rainfall': '7D_mean_rain'})
if 'longitude' in df_predict.columns and 'latitude' in df_predict.columns:
    df_predict = df_predict.rename(columns={'longitude': 'lon', 'latitude': 'lat'})

# DAta Cleaning
df_predict['7D_mean_rain'] = df_predict['7D_mean_rain'] / 10

# Apply NDVI normalization
if df_predict['ndvi'].max() > 100:
    df_predict["ndvi"] = (df_predict["ndvi"] / 10000).clip(-1, 1)

# Apply clipping to rainfall data
if '7D_mean_rain' in df_predict.columns:
    df_predict["7D_mean_rain"] = df_predict["7D_mean_rain"].clip(0, 300)
if '24H_rain' in df_predict.columns:
    df_predict["24H_rain"] = df_predict["24H_rain"].clip(0, 150)


# Prediction

features = [
    "elevation", "slope", "aspect", "landcover_class",
    "7D_mean_rain", "24H_rain", "ndvi"
]

# Ensure the data frame only contains the required features
X = df_predict[features].astype(float)
X = X.fillna(X.median())

print(" Running prediction using XGBoost model...")
df_predict["landslide_probability"] = model.predict_proba(X)[:, 1]

# Risk
def risk_level(p):
    if p >= 0.9:
        return "High"
    elif p >= 0.5:
        return "Moderate"
    else:
        return "Low"

df_predict["risk_level"] = df_predict["landslide_probability"].apply(risk_level)

# Output
df_predict.to_csv(OUTPUT_PATH, index=False)

print("\n--- Prediction Complete ---")
print(" Grid points analyzed:", len(df_predict))
print(" Final results saved at:", OUTPUT_PATH)
print("\nRisk Distribution:")
print(df_predict['risk_level'].value_counts(normalize=True))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Loading trained model and input data...
 Running prediction using XGBoost model...

--- Prediction Complete ---
 Grid points analyzed: 100
 Final results saved at: /content/drive/MyDrive/final_project/landslide_prediction/dataB_prediction(test)_ver4.csv

Risk Distribution:
risk_level
Low         0.49
High        0.39
Moderate    0.12
Name: proportion, dtype: float64
